When building agents that can leverage specialized expertise, we have a powerful tool at our disposal: the ability to have the LLM adopt different expert personas to provide specialized insights using the **Persona Pattern**. The `prompt_expert` tool serves as our foundation, allowing us to create interactions with experts who have specific backgrounds, expertise, and perspectives. Let’s explore how to build specialized expert tools for a software development workflow.

First, let’s look at our base expert consultation tool:

In [ ]:
@register_tool()
def prompt_expert(action_context: ActionContext, description_of_expert: str, prompt: str) -> str:
    """
    Generate a response from an expert persona.

    The expert's background and specialization should be thoroughly described to ensure
    responses align with their expertise. The prompt should be focused on topics within
    their domain of knowledge.

    Args:
        description_of_expert: Detailed description of the expert's background and expertise
        prompt: The specific question or task for the expert

    Returns:
        The expert's response
    """
    generate_response = action_context.get("llm")
    response = generate_response(Prompt(messages=[
        {"role": "system",
         "content": f"Act as the following expert and respond accordingly: {description_of_expert}"},
        {"role": "user", "content": prompt}
    ]))
    return response

**Prebuilt Experts**

Now, let’s build specialized expert tools for different aspects of our development process. Each tool directly uses prompt_expert with carefully crafted expert descriptions and focused prompts:

In [ ]:
# senior technical writer

@register_tool(tags=["documentation"])
def generate_technical_documentation(action_context: ActionContext, code_or_feature: str) -> str:
    """
    Generate technical documentation by consulting a senior technical writer.
    This expert focuses on creating clear, comprehensive documentation for developers.

    Args:
        code_or_feature: The code or feature to document
    """
    return prompt_expert(
        action_context=action_context,
        description_of_expert="""
        You are a senior technical writer with 15 years of experience in software documentation.
        You have particular expertise in:
        - Writing clear and precise API documentation
        - Explaining complex technical concepts to developers
        - Documenting implementation details and integration points
        - Creating code examples that illustrate key concepts
        - Identifying and documenting important caveats and edge cases

        Your documentation is known for striking the perfect balance between completeness
        and clarity. You understand that good technical documentation serves as both
        a reference and a learning tool.
        """,
        prompt=f"""
        Please create comprehensive technical documentation for the following code or feature:

        {code_or_feature}

        Your documentation should include:
        1. A clear overview of the feature's purpose and functionality
        2. Detailed explanation of the implementation approach
        3. Key interfaces and integration points
        4. Usage examples with code snippets
        5. Important considerations and edge cases
        6. Performance implications if relevant

        Focus on providing information that developers need to effectively understand
        and work with this code.
        """
    )

# senior QA engineer

@register_tool(tags=["testing"])
def design_test_suite(action_context: ActionContext, feature_description: str) -> str:
    """
    Design a comprehensive test suite by consulting a senior QA engineer.
    This expert focuses on creating thorough test coverage with attention to edge cases.

    Args:
        feature_description: Description of the feature to test
    """
    return prompt_expert(
        action_context=action_context,
        description_of_expert="""
        You are a senior QA engineer with 12 years of experience in test design and automation.
        Your expertise includes:
        - Comprehensive test strategy development
        - Unit, integration, and end-to-end testing
        - Performance and stress testing
        - Security testing considerations
        - Test automation best practices

        You are particularly skilled at identifying edge cases and potential failure modes
        that others might miss. Your test suites are known for their thoroughness and
        their ability to catch issues early in the development cycle.
        """,
        prompt=f"""
        Please design a comprehensive test suite for the following feature:

        {feature_description}

        Your test design should cover:
        1. Unit tests for individual components
        2. Integration tests for component interactions
        3. End-to-end tests for critical user paths
        4. Performance test scenarios if relevant
        5. Edge cases and error conditions
        6. Test data requirements

        For each test category, provide:
        - Specific test scenarios
        - Expected outcomes
        - Important edge cases to consider
        - Potential testing challenges
        """
    )

# senior software architect

@register_tool(tags=["code_quality"])
def perform_code_review(action_context: ActionContext, code: str) -> str:
    """
    Review code and suggest improvements by consulting a senior software architect.
    This expert focuses on code quality, architecture, and best practices.

    Args:
        code: The code to review
    """
    return prompt_expert(
        action_context=action_context,
        description_of_expert="""
        You are a senior software architect with 20 years of experience in code review
        and software design. Your expertise includes:
        - Software architecture and design patterns
        - Code quality and maintainability
        - Performance optimization
        - Scalability considerations
        - Security best practices

        You have a talent for identifying subtle design issues and suggesting practical
        improvements that enhance code quality without over-engineering.
        """,
        prompt=f"""
        Please review the following code and provide detailed improvement suggestions:

        {code}

        Consider and address:
        1. Code organization and structure
        2. Potential design pattern applications
        3. Performance optimization opportunities
        4. Error handling completeness
        5. Edge case handling
        6. Maintainability concerns

        For each suggestion:
        - Explain the current issue
        - Provide the rationale for change
        - Suggest specific improvements
        - Note any trade-offs to consider
        """
    )

# senior product marketing manager

@register_tool(tags=["communication"])
def write_feature_announcement(action_context: ActionContext,
                             feature_details: str,
                             audience: str) -> str:
    """
    Write a feature announcement by consulting a product marketing expert.
    This expert focuses on clear communication of technical features to different audiences.

    Args:
        feature_details: Technical details of the feature
        audience: Target audience for the announcement (e.g., "technical", "business")
    """
    return prompt_expert(
        action_context=action_context,
        description_of_expert="""
        You are a senior product marketing manager with 12 years of experience in
        technical product communication. Your expertise includes:
        - Translating technical features into clear value propositions
        - Crafting compelling product narratives
        - Adapting messaging for different audience types
        - Building excitement while maintaining accuracy
        - Creating clear calls to action

        You excel at finding the perfect balance between technical accuracy and
        accessibility, ensuring your communications are both precise and engaging.
        """,
        prompt=f"""
        Please write a feature announcement for the following feature:

        {feature_details}

        This announcement is intended for a {audience} audience.

        Your announcement should include:
        1. A compelling introduction
        2. Clear explanation of the feature
        3. Key benefits and use cases
        4. Technical details (adapted to audience)
        5. Implementation requirements
        6. Next steps or call to action

        Ensure the tone and technical depth are appropriate for a {audience} audience.
        Focus on conveying both the value and the practical implications of this feature.
        """
    )

# Dynamic vs. Static Expertise
The persona pattern can be implemented in two complementary ways: through pre-defined persona tools (static expertise) or through dynamically created personas (dynamic expertise). Each approach has distinct advantages and use cases.

## Static Expertise (Pre-defined Persona Tools)
Static expertise, as demonstrated in our code examples, involves creating dedicated tools for specific persona roles with carefully crafted descriptions and prompts. This approach:

1. Ensures Consistency: The persona’s background, approach, and focus areas remain consistent across interactions.
2. Enables Optimization: Persona descriptions and prompts can be refined over time to maximize effectiveness.
3. Simplifies Orchestration: The agent has a clear menu of available personas to consult.
4. Supports Validation: Static personas can be thoroughly tested to ensure they provide reliable guidance.

## Dynamic Expertise (On-the-fly Persona Creation)
Dynamic expertise involves creating new personas as needed based on the specific requirements of a task. This approach:

1. Offers Unlimited Specialization: The agent can create hyper-specialized personas for niche domains that weren’t anticipated during development.
2. Adapts to Novel Situations: When facing unprecedented challenges, the agent can design personas specifically tailored to those situations.
3. Enables Progressive Refinement: The agent can iteratively refine persona descriptions based on the results of initial consultations.
4. Supports Exploratory Problem-Solving: For ill-defined problems, the agent can create multiple persona perspectives to explore different solution paths.



Here’s how dynamic expertise might be implemented:

In [ ]:
@register_tool()
def create_and_consult_expert(action_context: ActionContext,
                             expertise_domain: str,
                             problem_description: str) -> str:
    """
    Dynamically create and consult an expert persona based on the specific domain and problem.

    Args:
        expertise_domain: The specific domain of expertise needed
        problem_description: Detailed description of the problem to be solved

    Returns:
        The expert's insights and recommendations
    """
    # Step 1: Dynamically generate a persona description
    persona_description_prompt = f"""
    Create a detailed description of an expert in {expertise_domain} who would be
    ideally suited to address the following problem:

    {problem_description}

    Your description should include:
    - The expert's background and experience
    - Their specific areas of specialization within {expertise_domain}
    - Their approach to problem-solving
    - The unique perspective they bring to this type of challenge
    """

    generate_response = action_context.get("llm")
    persona_description = generate_response(Prompt(messages=[
        {"role": "user", "content": persona_description_prompt}
    ]))

    # Step 2: Generate a specialized consultation prompt
    consultation_prompt_generator = f"""
    Create a detailed consultation prompt for an expert in {expertise_domain}
    addressing the following problem:

    {problem_description}

    The prompt should guide the expert to provide comprehensive insights and
    actionable recommendations specific to this problem.
    """

    consultation_prompt = generate_response(Prompt(messages=[
        {"role": "user", "content": consultation_prompt_generator}
    ]))

    # Step 3: Consult the dynamically created persona
    return prompt_expert(
        action_context=action_context,
        description_of_expert=persona_description,
        prompt=consultation_prompt
    )

A sophisticated implementation might combine both approaches, using pre-defined personas for common domains while dynamically creating personas for specialized or unexpected scenarios.

##Chain of Expertise

The persona pattern enables the creation of expertise chains, where outputs from one persona become inputs to another, creating sophisticated workflows that mirror real-world collaborative processes.

This approach enables:

1. Progressive Refinement: Ideas can evolve through successive persona consultations, with each persona adding value based on their domain knowledge.

2. Cross-Domain Integration: Complex problems that span multiple domains can be addressed by systematically consulting personas in each relevant domain.

3. Specialized Workflow Stages: Different stages of a workflow (design, implementation, testing, documentation, etc.) can be handled by different personas with specialized knowledge for each stage.

4. Checks and Balances: Personas can review each other’s work, providing a form of quality control similar to how different departments in an organization review projects before they’re finalized.

Implementation considerations for expertise chains include:

1. Information Transfer: Ensure that outputs from one persona contain all the information needed by the next persona in the chain.

2. Context Preservation: Maintain important context as information flows through the chain to prevent misunderstandings or loss of critical details.

3. Feedback Loops: Allow for circular references where later personas can send questions or suggestions back to earlier personas in the chain.

4. Conflict Resolution: Develop strategies for resolving conflicting recommendations from different personas in the chain.

Here’s a simplified example of an expertise chain for software development:

In [ ]:
def develop_feature(action_context: ActionContext, feature_request: str) -> dict:
    """
    Process a feature request through a chain of expert personas.
    """
    # Step 1: Product expert defines requirements
    requirements = prompt_expert(
        action_context,
        "product manager expert",
        f"Convert this feature request into detailed requirements: {feature_request}"
    )

    # Step 2: Architecture expert designs the solution
    architecture = prompt_expert(
        action_context,
        "software architect expert",
        f"Design an architecture for these requirements: {requirements}"
    )

    # Step 3: Developer expert implements the code
    implementation = prompt_expert(
        action_context,
        "senior developer expert",
        f"Implement code for this architecture: {architecture}"
    )

    # Step 4: QA expert creates test cases
    tests = prompt_expert(
        action_context,
        "QA engineer expert",
        f"Create test cases for this implementation: {implementation}"
    )

    # Step 5: Documentation expert creates documentation
    documentation = prompt_expert(
        action_context,
        "technical writer expert",
        f"Document this implementation: {implementation}"
    )

    return {
        "requirements": requirements,
        "architecture": architecture,
        "implementation": implementation,
        "tests": tests,
        "documentation": documentation
    }